# 1. Context

This notebook is created for loading & preprocessing Data for [Word2Vec Implementation](https://arxiv.org/abs/1301.3781). Scale of Data Processing will be reduced to fit consumer GPUs Contraints

# 2. Dataset in Scope